In [1]:
from param_def import chi,sigma,prop_gp,capacities_rate,prop_all_g_prefer,capacities,lambdas
import numpy as np
from util import welfare_metrics,student_by_col,bayes_update_grade,grades_gr_to_grades_col,utility_by_col,welfare_by_col,market_clear_noise_corr
from itertools import compress
# Import HyperOpt Library
from hyperopt import tpe, hp, fmin
import numpy as np
from BayesEst import bayesian_est,anal_cond_exp
from scipy.optimize import fsolve
from scipy.stats import norm, multivariate_normal
from param_def import chi,sigma,lambdas

In [2]:
print(capacities_rate)

[0.2, 0.2]


In [3]:
grade_estimated_gr = []
for i in range(len(chi)):
    grade_estimated_gr.append(np.load(f'grade_estimated_gr{i+1}.npy'))
grade_estimated = grades_gr_to_grades_col(grade_estimated_gr)
#%%
stud_pref = []
for i in range(len(chi)):
    stud_pref.append(np.load(f'stud_pref_gr{i+1}.npy').tolist())

In [4]:
def cdf(x, sigma):
    return norm.cdf(x, scale = sigma)

In [5]:
def cdfm(x, y, sigma, rho):
    return multivariate_normal([0, 0], [[sigma**2, sigma**2 * rho], [sigma**2 * rho, sigma**2]]).cdf(np.array([x,y]))

In [6]:
def cdfmsi(x, y, sigma, rho):
    return norm.cdf(y, scale = sigma) - multivariate_normal([0, 0], [[sigma**2, sigma**2 * rho], [sigma**2 * rho, sigma**2]]).cdf(np.array([x,y]))

In [7]:
def cdfmis(x, y, sigma, rho):
    return norm.cdf(x, scale = sigma) - multivariate_normal([0, 0], [[sigma**2, sigma**2 * rho], [sigma**2 * rho, sigma**2]]).cdf(np.array([x,y]))

In [8]:
def market_clear(Pa, Pb, prop, capA, capB, prefi, prefii, sigmai, sigmaii, cori, corii):
    f1 = prop*prefi*(1 - cdf(Pa, sigmai)) + (1 - prop)*prefii*(1 - cdf(Pa, sigmaii)) + prop*(1 - prefi)*cdfmsi(Pa, Pb, sigmai, cori) + (1 -prop)*(1 - prefii)*cdfmsi(Pa, Pb, sigmaii, corii) - capA
    f2 = prop*(1 - prefi)*(1 - cdf(Pb, sigmai)) + (1 - prop)*(1 - prefii)*(1 - cdf(Pb, sigmaii)) + prop*prefi*cdfmis(Pa, Pb, sigmai, cori) + (1 -prop)*prefii*cdfmis(Pa, Pb, sigmaii, corii) - capB
    return f1, f2

In [9]:
def objective_bads(params):
    Pa,Pb = params
    f1,f2 = market_clear(Pa,Pb,prop_gp[0],capacities_rate[0],capacities_rate[1],
                  prop_all_g_prefer[0],prop_all_g_prefer[1],sigma_i,sigma_ii,cor_i,cor_ii)
    # return np.abs(f1) + np.abs(f2)
    return f1**2+f2**2

In [10]:
# def solv_mc(prop, capA, capB, prefi, prefii, sigmai, sigmaii, cori, corii):
#     func = lambda P: market_clear(P[0], P[1], prop, capA, capB, prefi, prefii, sigmai, sigmaii, cori, corii)
#     return fsolve(func, x0 = np.array([0, 0]))

# P_A,P_B = solv_mc(prop_gp[0],capacities_rate[0],capacities_rate[1],
#                   prop_all_g_prefer[0],prop_all_g_prefer[1],sigma_i,sigma_ii,cor_i,cor_ii)


In [11]:
sigma_i = np.sqrt(chi[0]**2 + sigma[0]**2)
sigma_ii = np.sqrt(chi[1]**2 + sigma[1]**2)
cor_i = (chi[0]**2 + lambdas[0]**2)/sigma_i**2
cor_ii = (chi[1]**2 + lambdas[1]**2)/sigma_ii**2

In [12]:
from pybads import BADS

lower_bounds = np.array([-10, -10])
upper_bounds = np.array([10, 10])
plausible_lower_bounds = np.array([-5, -5])
plausible_upper_bounds = np.array([5, 5])
x0 = np.array([0, 0]);        # Starting point
# %%
bads = BADS(objective_bads, x0, lower_bounds, upper_bounds, plausible_lower_bounds, plausible_upper_bounds)
optimize_result = bads.optimize()

# %%
x_min = optimize_result['x']
fval = optimize_result['fval']
print(f"BADS minimum at: x_min = {x_min.flatten()}, fval = {fval:.4g}")
print(f"total f-count: {optimize_result['func_count']}, time: {round(optimize_result['total_time'], 2)} s")

Beginning optimization of a DETERMINISTIC objective function

 Iteration    f-count         f(x)           MeshScale          Method             Actions
     0           2       0.0434661               1                                 Uncertainty test
     0           6       0.0304515               1         Initial mesh            Initial points
     0          10      0.00547004             0.5         Refine grid             Train
     1          12      0.00416473             0.5     Incremental search (ES-wcm)        
     1          18      0.00416473            0.25         Refine grid             Train
     2          19     0.000186839            0.25     Incremental search (ES-ell)        
     2          20     2.58606e-05            0.25     Incremental search (ES-wcm)        
     2          21     1.03538e-05            0.25     Incremental search (ES-wcm)        
     2          26     1.03538e-05           0.125         Refine grid             
     3          29     

In [13]:
cutoff_values = list(x_min)

In [14]:
welfare_metrics(cutoff_values,grade_estimated,stud_pref)

Proportion of students in group 0 with no offer 0.62
Proportion of students in group 0 with only a second preference offer 0.13
Proportion of students in group 0 with first choice offer 0.25
Proportion of students in group 1 with no offer 0.59
Proportion of students in group 1 with only a second preference offer 0.16
Proportion of students in group 1 with first choice offer 0.25


In [15]:
student_by_col(cutoff_values,grade_estimated,stud_pref)

Number of student admitted to college 1 in group 0: 1477
Number of student admitted to college 2 in group 0: 1541
Number of student admitted to college 1 in group 1: 2432
Number of student admitted to college 2 in group 1: 2507


(3909, 4048)

In [16]:
utility_by_col(cutoff_values,grade_estimated,stud_pref)

Total A utility: 27437.41813871766
A utility for student with first choice: 14775.6579629956
A utility for student with second choice: 12661.760175722062
Total B utility: 30082.659215914086
B utility for student with first choice: 22463.00142907581
B utility for student with second choice: 7619.657786838274


True

In [17]:
welfare_by_col(cutoff_values,grade_estimated,stud_pref)

Proportion of student admitted to A with first choice: 0.5085
Proportion of student admitted to A with second choice: 0.46875
Proportion of student admitted to B with first choice: 0.74
Proportion of student admitted to B with second choice: 0.272


True

In [18]:
print(capacities)

[4000, 4000]


In [19]:
import pandas as pd
df1 = pd.DataFrame({'A':grade_estimated[0][0],'B':grade_estimated[1][0],'pref':np.array(stud_pref[0]).T[0]})
df2 = pd.DataFrame({'A':grade_estimated[0][1],'B':grade_estimated[1][1],'pref':np.array(stud_pref[1]).T[0]})

Partially update

In [20]:
def objective_bads(params):
    Pa,Pb = params
    f1,f2 = market_clear_noise_corr(Pa, Pb, grade_estimated, stud_pref, prop_gp[0], capacities_rate[0], capacities_rate[1], prop_all_g_prefer[0], prop_all_g_prefer[1], sigma_i, sigma_ii, cor_i, cor_ii,chi,sigma,lambdas,bayes='right_partial')
    return np.abs(f1) + np.abs(f2)
    # return f1**2+f2**2
# %%
lower_bounds = np.array([-10, -10])
upper_bounds = np.array([10, 10])
plausible_lower_bounds = np.array([-5, -5])
plausible_upper_bounds = np.array([5, 5])
x0 = np.array([0, 0]);        # Starting point
# %%
bads = BADS(objective_bads, x0, lower_bounds, upper_bounds, plausible_lower_bounds, plausible_upper_bounds)
optimize_result = bads.optimize()

# %%
x_min = optimize_result['x']
fval = optimize_result['fval']
print(f"BADS minimum at: x_min = {x_min.flatten()}, fval = {fval:.4g}")
print(f"total f-count: {optimize_result['func_count']}, time: {round(optimize_result['total_time'], 2)} s")

Beginning optimization of a DETERMINISTIC objective function

 Iteration    f-count         f(x)           MeshScale          Method             Actions
     0           2         0.22291               1                                 Uncertainty test
     0           6         0.22291               1         Initial mesh            Initial points
     0          10         0.22291             0.5         Refine grid             Train
     1          11        0.077655             0.5     Incremental search (ES-wcm)        
     1          12       0.0556941             0.5     Incremental search (ES-wcm)        
     1          18       0.0556941            0.25         Refine grid             Train
     2          20       0.0312797            0.25     Incremental search (ES-wcm)        
     2          21       0.0303288            0.25     Incremental search (ES-ell)        
     2          26       0.0303288           0.125         Refine grid             
     3          27     

In [21]:
new_cutoff_values = list(x_min)
new_cutoff_values

[np.float64(4.064505435526371), np.float64(1.0284804925322533)]

In [22]:
updated_grade_estimated = bayes_update_grade(new_cutoff_values[0],new_cutoff_values[1],grade_estimated,stud_pref,chi,sigma,lambdas,bayes_type='right_partial')

In [23]:
##    
# grade_estimated should have the following form 
# [[[-3.62656622994722, 2.1811620120023885, 
# -11.563939588668866, 9.522766463940787], 
# [-2.140950614292475, -3.1545339753546977, 
# 0.2686527056082677, -0.6574626041532636, 
# -2.252868772168457, 1.8937660992816987]], 
# [[-5.268780813142571, -0.8191620654894043, 
# -9.60125782299109, 7.9126466724151605], 
# [-3.9756947771758844, -2.6409442009387023, 
# 4.086647894727917, -1.357128391535272, 
# 0.8661128454577303, 0.9586986061854232]]] 
# with 2 groups, 2 colleges, group 1 has 4 students, 
# group 2 has 6 students.
# updated_grade_estimate should have the same form as grade_estimated

In [24]:
welfare_metrics(new_cutoff_values,updated_grade_estimated,stud_pref)

Proportion of students in group 0 with no offer 0.62
Proportion of students in group 0 with only a second preference offer 0.20
Proportion of students in group 0 with first choice offer 0.18
Proportion of students in group 1 with no offer 0.59
Proportion of students in group 1 with only a second preference offer 0.23
Proportion of students in group 1 with first choice offer 0.18


In [25]:
student_by_col(new_cutoff_values,updated_grade_estimated,stud_pref)

Number of student admitted to college 1 in group 0: 1555
Number of student admitted to college 2 in group 0: 1505
Number of student admitted to college 1 in group 1: 2449
Number of student admitted to college 2 in group 1: 2490


(4004, 3995)

In [26]:
utility_by_col(new_cutoff_values,updated_grade_estimated,stud_pref)

Total A utility: 29419.51885006898
A utility for student with first choice: 14201.351780092724
A utility for student with second choice: 15218.167069976256
Total B utility: 13675.039773397988
B utility for student with first choice: 2967.011857884886
B utility for student with second choice: 10708.027915513103


True

In [27]:
welfare_by_col(new_cutoff_values,updated_grade_estimated,stud_pref)

Proportion of student admitted to A with first choice: 0.4715
Proportion of student admitted to A with second choice: 0.5295
Proportion of student admitted to B with first choice: 0.4325
Proportion of student admitted to B with second choice: 0.56625


True

Update all

In [45]:
new_cutoff_values = [1.57124967,0.40021554]
new_cutoff_values

[1.57124967, 0.40021554]

In [46]:
updated_grade_estimated = bayes_update_grade(new_cutoff_values[0],new_cutoff_values[1],grade_estimated,stud_pref,chi,sigma,lambdas,bayes_type='right_all')

In [47]:
welfare_metrics(new_cutoff_values,updated_grade_estimated,stud_pref)

Proportion of students in group 0 with no offer 0.59
Proportion of students in group 0 with only a second preference offer 0.14
Proportion of students in group 0 with first choice offer 0.27
Proportion of students in group 1 with no offer 0.61
Proportion of students in group 1 with only a second preference offer 0.12
Proportion of students in group 1 with first choice offer 0.27


In [48]:
student_by_col(new_cutoff_values,updated_grade_estimated,stud_pref)

Number of student admitted to college 1 in group 0: 1631
Number of student admitted to college 2 in group 0: 1624
Number of student admitted to college 1 in group 1: 2414
Number of student admitted to college 2 in group 1: 2319
4045 3943


(4045, 3943)

In [49]:
utility_by_col(new_cutoff_values,updated_grade_estimated,stud_pref)

Total A utility: 12778.985772298567
A utility for student with first choice: 7897.849522773078
A utility for student with second choice: 4881.136249525489
Total B utility: 4135.683811191746
B utility for student with first choice: 3331.118058670191
B utility for student with second choice: 804.5657525215552


True

In [50]:
welfare_by_col(new_cutoff_values,updated_grade_estimated,stud_pref)

Proportion of student admitted to A with first choice: 0.59375
Proportion of student admitted to A with second choice: 0.4175
Proportion of student admitted to B with first choice: 0.767
Proportion of student admitted to B with second choice: 0.21875


True